# SAXS Structure Analysis with XPCS Toolkit

This tutorial covers comprehensive Small-Angle X-ray Scattering (SAXS) analysis techniques using the XPCS Toolkit. You'll learn to analyze particle size, shape, interactions, and structural organization from scattering data.

## Learning Objectives

By the end of this tutorial, you will:
1. Understand SAXS scattering theory and interpretation
2. Perform Guinier analysis for size determination
3. Apply Porod analysis for surface characterization
4. Analyze particle interactions and structure factors
5. Work with both 1D and 2D SAXS data
6. Extract quantitative structural parameters


In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize, special

# XPCS Toolkit imports

# Configure plotting
%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 11
plt.rcParams["lines.markersize"] = 4

print("SAXS Structure Analysis Tutorial")
print("XPCS Toolkit loaded successfully!")

## SAXS Theory Fundamentals

### Basic Scattering Equation

The scattered intensity I(q) for a system of particles is:
$$I(q) = n \langle|F(q)|^2\rangle S(q)$$

Where:
- **n**: Number density of scatterers
- **F(q)**: Form factor (single particle scattering)
- **S(q)**: Structure factor (inter-particle correlations)
- **q = 4π sin(θ/2)/λ**: Momentum transfer

In [ ]:
# Define common form factors


def sphere_form_factor(q, R):
    """
    Form factor for solid spheres.

    Parameters:
    -----------
    q : array
        Momentum transfer (Å⁻¹)
    R : float
        Sphere radius (Å)

    Returns:
    --------
    F : array
        Form factor amplitude
    """
    qR = q * R
    # Handle q=0 case
    qR = np.where(qR == 0, 1e-10, qR)

    F = 3 * (np.sin(qR) - qR * np.cos(qR)) / (qR**3)
    return F


def gaussian_chain_form_factor(q, Rg):
    """
    Form factor for Gaussian polymer chains.

    Parameters:
    -----------
    q : array
        Momentum transfer (Å⁻¹)
    Rg : float
        Radius of gyration (Å)

    Returns:
    --------
    F : array
        Form factor amplitude
    """
    x = (q * Rg) ** 2
    # Debye function
    F = 2 * (np.exp(-x) - 1 + x) / (x**2)
    # Handle x=0 case
    F = np.where(x == 0, 1.0, F)
    return F


def cylinder_form_factor(q, R, L):
    """
    Form factor for cylinders (rod-like particles).

    This is a simplified version - full calculation requires
    integration over all orientations.
    """
    qR = q * R
    qL = q * L

    # Simplified approximation for randomly oriented cylinders
    F_cross = 2 * special.j1(qR) / qR  # Cross-section
    F_length = np.sin(qL / 2) / (qL / 2)  # Length

    # Handle q=0 cases
    F_cross = np.where(qR == 0, 1.0, F_cross)
    F_length = np.where(qL == 0, 1.0, F_length)

    return F_cross * F_length


# Generate example form factors
q_range = np.logspace(-2, 0, 100)  # 0.01 to 1.0 Å⁻¹

# Different particle types
F_sphere_small = sphere_form_factor(q_range, R=20)  # 20 Å sphere
F_sphere_large = sphere_form_factor(q_range, R=50)  # 50 Å sphere
F_chain = gaussian_chain_form_factor(q_range, Rg=30)  # Polymer chain
F_cylinder = cylinder_form_factor(q_range, R=10, L=100)  # Rod

plt.figure(figsize=(15, 10))

# Form factors
plt.subplot(2, 3, 1)
plt.loglog(q_range, F_sphere_small**2, "b-", linewidth=2, label="Small sphere (R=20Å)")
plt.loglog(q_range, F_sphere_large**2, "r-", linewidth=2, label="Large sphere (R=50Å)")
plt.xlabel("q (Å⁻¹)")
plt.ylabel("|F(q)|²")
plt.title("Sphere Form Factors")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 3, 2)
plt.loglog(q_range, F_chain**2, "g-", linewidth=2, label="Gaussian chain (Rg=30Å)")
plt.loglog(q_range, F_cylinder**2, "m-", linewidth=2, label="Cylinder (R=10Å, L=100Å)")
plt.xlabel("q (Å⁻¹)")
plt.ylabel("|F(q)|²")
plt.title("Chain and Rod Form Factors")
plt.legend()
plt.grid(True, alpha=0.3)

# Scattering intensities (with arbitrary normalization)
I_sphere_small = 1000 * F_sphere_small**2 + 0.1  # Add background
I_sphere_large = 1000 * F_sphere_large**2 + 0.1
I_chain = 500 * F_chain**2 + 0.1
I_cylinder = 800 * F_cylinder**2 + 0.1

plt.subplot(2, 3, 3)
plt.loglog(q_range, I_sphere_small, "b-", linewidth=2, label="Small spheres")
plt.loglog(q_range, I_sphere_large, "r-", linewidth=2, label="Large spheres")
plt.xlabel("q (Å⁻¹)")
plt.ylabel("I(q)")
plt.title("Scattering Intensities - Spheres")
plt.legend()
plt.grid(True, alpha=0.3)

# Kratky plots (q²I vs q)
plt.subplot(2, 3, 4)
plt.plot(q_range, q_range**2 * I_sphere_small, "b-", linewidth=2, label="Small spheres")
plt.plot(q_range, q_range**2 * I_sphere_large, "r-", linewidth=2, label="Large spheres")
plt.xlabel("q (Å⁻¹)")
plt.ylabel("q²I(q)")
plt.title("Kratky Plot - Spheres")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 3, 5)
plt.plot(q_range, q_range**2 * I_chain, "g-", linewidth=2, label="Gaussian chain")
plt.plot(q_range, q_range**2 * I_cylinder, "m-", linewidth=2, label="Cylinder")
plt.xlabel("q (Å⁻¹)")
plt.ylabel("q²I(q)")
plt.title("Kratky Plot - Chains & Rods")
plt.legend()
plt.grid(True, alpha=0.3)

# Porod plots (q⁴I vs q)
plt.subplot(2, 3, 6)
plt.loglog(
    q_range, q_range**4 * I_sphere_small, "b-", linewidth=2, label="Small spheres"
)
plt.loglog(
    q_range, q_range**4 * I_sphere_large, "r-", linewidth=2, label="Large spheres"
)
plt.axhline(
    y=1000 * 2 * np.pi, color="k", linestyle="--", alpha=0.7, label="Porod limit"
)
plt.xlabel("q (Å⁻¹)")
plt.ylabel("q⁴I(q)")
plt.title("Porod Plot - Surface Scattering")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key form factor characteristics:")
print("- Spheres: Oscillations with period Δq ≈ π/R")
print("- Chains: Plateau in Kratky plot for flexible polymers")
print("- Rods: q⁻¹ decay at intermediate q")
print("- All particles: q⁻⁴ Porod behavior at high q")

## Guinier Analysis

### Small-q Approximation

At small q (qR_g < 1), the intensity follows:
$$\ln I(q) = \ln I(0) - \frac{q^2 R_g^2}{3}$$

The radius of gyration R_g can be extracted from the slope.

In [ ]:
def guinier_analysis(q, I, q_max_factor=1.3):
    """
    Perform Guinier analysis to extract radius of gyration.

    Parameters:
    -----------
    q : array
        Momentum transfer values
    I : array
        Scattering intensity values
    q_max_factor : float
        Maximum qRg for valid Guinier region (typically 1.0-1.3)

    Returns:
    --------
    results : dict
        Dictionary containing Rg, I0, fit quality, and valid q-range
    """
    results = {}

    # Remove zero or negative intensities
    valid_mask = (I > 0) & (q > 0)
    q_valid = q[valid_mask]
    I_valid = I[valid_mask]

    if len(q_valid) < 5:
        print("Not enough valid data points for Guinier analysis")
        return None

    # Initial estimate of Rg from first few points
    n_initial = min(10, len(q_valid) // 2)
    q_initial = q_valid[:n_initial]
    I_initial = I_valid[:n_initial]

    try:
        # Linear fit to ln(I) vs q²
        q2_initial = q_initial**2
        ln_I_initial = np.log(I_initial)

        coeffs = np.polyfit(q2_initial, ln_I_initial, 1)
        slope_initial = coeffs[0]
        coeffs[1]

        # Extract Rg from slope: slope = -Rg²/3
        if slope_initial < 0:
            Rg_estimate = np.sqrt(-3 * slope_initial)
        else:
            print("Warning: Positive slope in Guinier region - may not be valid")
            Rg_estimate = 20.0  # Default guess

        # Define valid Guinier range: qRg < q_max_factor
        q_max_guinier = q_max_factor / Rg_estimate
        guinier_mask = q_valid <= q_max_guinier

        if np.sum(guinier_mask) < 3:
            print(
                f"Warning: Very few points ({np.sum(guinier_mask)}) in Guinier region"
            )
            return None

        # Final fit with proper Guinier range
        q_guinier = q_valid[guinier_mask]
        I_guinier = I_valid[guinier_mask]
        q2_guinier = q_guinier**2
        ln_I_guinier = np.log(I_guinier)

        # Weighted linear fit (higher weight to lower q)
        weights = 1.0 / q_guinier  # Lower q gets higher weight
        weights = weights / np.sum(weights)

        coeffs_final = np.polyfit(q2_guinier, ln_I_guinier, 1, w=weights)
        slope_final = coeffs_final[0]
        intercept_final = coeffs_final[1]

        # Calculate errors
        ln_I_fit = np.polyval(coeffs_final, q2_guinier)
        residuals = ln_I_guinier - ln_I_fit
        chi2 = np.sum(residuals**2)
        chi2_reduced = chi2 / (len(q_guinier) - 2)

        # Extract final parameters
        Rg_final = np.sqrt(-3 * slope_final) if slope_final < 0 else np.nan
        I0_final = np.exp(intercept_final)

        # Estimate parameter errors (simplified)
        std_residuals = np.std(residuals)
        Rg_error = (
            std_residuals * Rg_final / abs(slope_final)
            if not np.isnan(Rg_final)
            else np.nan
        )
        I0_error = std_residuals * I0_final

        results = {
            "Rg": Rg_final,
            "Rg_error": Rg_error,
            "I0": I0_final,
            "I0_error": I0_error,
            "slope": slope_final,
            "intercept": intercept_final,
            "chi2_reduced": chi2_reduced,
            "q_range": (q_guinier.min(), q_guinier.max()),
            "qRg_max": q_guinier.max() * Rg_final if not np.isnan(Rg_final) else np.nan,
            "n_points": len(q_guinier),
            "q_fit": q_guinier,
            "I_fit": np.exp(ln_I_fit),
            "valid_fit": not np.isnan(Rg_final) and Rg_final > 0,
        }

    except Exception as e:
        print(f"Guinier analysis failed: {e}")
        return None

    return results


# Test Guinier analysis with synthetic data
q_test = np.logspace(-2, 0, 50)

# Generate data for different particle sizes
particles = [
    {"name": "Small spheres", "R": 15, "color": "blue"},
    {"name": "Medium spheres", "R": 30, "color": "red"},
    {"name": "Large spheres", "R": 50, "color": "green"},
]

plt.figure(figsize=(16, 12))
guinier_results = {}

for i, particle in enumerate(particles):
    R = particle["R"]
    Rg_true = np.sqrt(3 / 5) * R  # For solid sphere: Rg = √(3/5) * R

    # Generate synthetic SAXS data
    F = sphere_form_factor(q_test, R)
    I_clean = 1000 * F**2 + 1  # Add small background

    # Add realistic noise
    noise_level = 0.05
    I_noisy = I_clean * (1 + noise_level * np.random.normal(0, 1, len(q_test)))
    I_noisy = np.maximum(I_noisy, 0.1)  # Ensure positive values

    # Perform Guinier analysis
    guinier_result = guinier_analysis(q_test, I_noisy)
    guinier_results[particle["name"]] = guinier_result

    # Plot results
    row = i // 2
    col = (i % 2) * 2

    # Guinier plot (ln I vs q²)
    plt.subplot(3, 4, row * 4 + col + 1)
    plt.semilogy(q_test, I_noisy, "o", color=particle["color"], alpha=0.6, markersize=3)

    if guinier_result and guinier_result["valid_fit"]:
        plt.semilogy(
            guinier_result["q_fit"],
            guinier_result["I_fit"],
            "-",
            color="black",
            linewidth=2,
            label="Guinier fit",
        )

        # Mark Guinier region
        q_max_guinier = guinier_result["q_range"][1]
        plt.axvline(
            q_max_guinier,
            color="red",
            linestyle="--",
            alpha=0.7,
            label=f"qRg={guinier_result['qRg_max']:.1f}",
        )

    plt.xlabel("q (Å⁻¹)")
    plt.ylabel("I(q)")
    plt.title(f"{particle['name']} (R={R}Å)")
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=8)

    # Linear Guinier plot (ln I vs q²)
    plt.subplot(3, 4, row * 4 + col + 2)
    plt.plot(
        q_test**2,
        np.log(I_noisy),
        "o",
        color=particle["color"],
        alpha=0.6,
        markersize=3,
    )

    if guinier_result and guinier_result["valid_fit"]:
        q2_fit = guinier_result["q_fit"] ** 2
        ln_I_fit = np.log(guinier_result["I_fit"])
        plt.plot(q2_fit, ln_I_fit, "-", color="black", linewidth=2)

        # Show fit equation
        plt.text(
            0.05,
            0.95,
            f"Rg = {guinier_result['Rg']:.1f} ± {guinier_result['Rg_error']:.1f} Å\n"
            f"True Rg = {Rg_true:.1f} Å\n"
            f"χ²r = {guinier_result['chi2_reduced']:.2f}",
            transform=plt.gca().transAxes,
            fontsize=8,
            verticalalignment="top",
            bbox={"boxstyle": "round,pad=0.3", "facecolor": "white", "alpha": 0.8},
        )

    plt.xlabel("q² (Å⁻²)")
    plt.ylabel("ln I(q)")
    plt.title("Guinier Plot")
    plt.grid(True, alpha=0.3)

# Summary comparison table
plt.subplot(3, 4, 9)
plt.axis("off")

summary_data = []
for particle in particles:
    R_true = particle["R"]
    Rg_true = np.sqrt(3 / 5) * R_true
    result = guinier_results[particle["name"]]

    if result and result["valid_fit"]:
        Rg_measured = result["Rg"]
        Rg_error = result["Rg_error"]
        error_percent = abs(Rg_measured - Rg_true) / Rg_true * 100
        summary_data.append(
            [
                particle["name"],
                f"{Rg_true:.1f}",
                f"{Rg_measured:.1f}±{Rg_error:.1f}",
                f"{error_percent:.1f}%",
            ]
        )
    else:
        summary_data.append([particle["name"], f"{Rg_true:.1f}", "Fit failed", "-"])

table = plt.table(
    cellText=summary_data,
    colLabels=["Sample", "True Rg (Å)", "Measured Rg (Å)", "Error"],
    cellLoc="center",
    loc="center",
    bbox=[0, 0.3, 1, 0.6],
)
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 2)

plt.title("Guinier Analysis Summary", y=0.9)

plt.tight_layout()
plt.show()

print("\n" + "=" * 50)
print("GUINIER ANALYSIS RESULTS")
print("=" * 50)

for particle in particles:
    R_true = particle["R"]
    Rg_true = np.sqrt(3 / 5) * R_true
    result = guinier_results[particle["name"]]

    print(f"\n{particle['name']}:")
    print(f"  True radius: {R_true} Å")
    print(f"  True Rg: {Rg_true:.2f} Å")

    if result and result["valid_fit"]:
        print(f"  Measured Rg: {result['Rg']:.2f} ± {result['Rg_error']:.2f} Å")
        print(f"  Error: {abs(result['Rg'] - Rg_true) / Rg_true * 100:.1f}%")
        print(
            f"  Q-range: {result['q_range'][0]:.3f} to {result['q_range'][1]:.3f} Å⁻¹"
        )
        print(f"  qRg_max: {result['qRg_max']:.2f}")
        print(f"  χ²_reduced: {result['chi2_reduced']:.2f}")

        if result["qRg_max"] > 1.3:
            print("  ⚠ Warning: qRg_max > 1.3 may indicate invalid Guinier region")
        else:
            print("  ✓ Valid Guinier region")
    else:
        print("  ❌ Guinier fit failed")

## Porod Analysis

### High-q Behavior

At large q (qR >> 1), for particles with sharp interfaces:
$$I(q) = \frac{2\pi(\Delta\rho)^2 S}{q^4} + B$$

Where S is the total surface area and B is the background.

In [ ]:
def porod_analysis(q, I, q_min_factor=3.0):
    """
    Perform Porod analysis to extract surface area information.

    Parameters:
    -----------
    q : array
        Momentum transfer values
    I : array
        Scattering intensity values
    q_min_factor : float
        Minimum qRg for valid Porod region (typically 3.0-5.0)

    Returns:
    --------
    results : dict
        Dictionary containing Porod constant, background, and fit quality
    """
    results = {}

    # Remove zero or negative intensities
    valid_mask = (I > 0) & (q > 0)
    q_valid = q[valid_mask]
    I_valid = I[valid_mask]

    if len(q_valid) < 5:
        print("Not enough valid data points for Porod analysis")
        return None

    # Estimate Rg from Guinier analysis to define Porod region
    guinier_result = guinier_analysis(q_valid, I_valid)
    if guinier_result and guinier_result["valid_fit"]:
        Rg_estimate = guinier_result["Rg"]
    else:
        # Fallback: estimate from inflection point
        log_I = np.log(I_valid)
        d_log_I = np.gradient(log_I, q_valid)
        inflection_idx = np.argmin(d_log_I)
        q_inflection = q_valid[inflection_idx]
        Rg_estimate = 1.0 / q_inflection  # Rough estimate

    # Define Porod region: qRg > q_min_factor
    q_min_porod = q_min_factor / Rg_estimate
    porod_mask = q_valid >= q_min_porod

    if np.sum(porod_mask) < 3:
        print(f"Warning: Very few points ({np.sum(porod_mask)}) in Porod region")
        # Use the high-q half of the data
        n_half = len(q_valid) // 2
        porod_mask = np.zeros(len(q_valid), dtype=bool)
        porod_mask[n_half:] = True

    q_porod = q_valid[porod_mask]
    I_porod = I_valid[porod_mask]

    try:
        # Fit to Porod law: I = K/q⁴ + B
        def porod_function(q, K, B):
            return K / (q**4) + B

        # Initial guess
        K_guess = I_porod[0] * q_porod[0] ** 4
        B_guess = np.min(I_porod)

        popt, pcov = optimize.curve_fit(
            porod_function,
            q_porod,
            I_porod,
            p0=[K_guess, B_guess],
            bounds=([0, 0], [np.inf, np.max(I_porod)]),
        )

        K_fit, B_fit = popt
        K_err, B_err = np.sqrt(np.diag(pcov))

        # Calculate fit quality
        I_fit = porod_function(q_porod, K_fit, B_fit)
        residuals = I_porod - I_fit
        chi2 = np.sum(residuals**2)
        chi2_reduced = chi2 / (len(q_porod) - 2)

        results = {
            "K_porod": K_fit,
            "K_porod_error": K_err,
            "background": B_fit,
            "background_error": B_err,
            "chi2_reduced": chi2_reduced,
            "q_range": (q_porod.min(), q_porod.max()),
            "qRg_min": q_porod.min() * Rg_estimate,
            "n_points": len(q_porod),
            "q_fit": q_porod,
            "I_fit": I_fit,
            "Rg_estimate": Rg_estimate,
            "valid_fit": True,
        }

    except Exception as e:
        print(f"Porod analysis failed: {e}")
        return None

    return results


# Test Porod analysis
plt.figure(figsize=(15, 10))
porod_results = {}

for i, particle in enumerate(particles):
    R = particle["R"]

    # Generate synthetic SAXS data with extended q-range
    q_extended = np.logspace(-2, 0.5, 80)  # Extended to higher q
    F = sphere_form_factor(q_extended, R)
    I_clean = 1000 * F**2 + 2  # Add background

    # Add realistic noise
    noise_level = 0.03
    I_noisy = I_clean * (1 + noise_level * np.random.normal(0, 1, len(q_extended)))
    I_noisy = np.maximum(I_noisy, 0.1)

    # Perform Porod analysis
    porod_result = porod_analysis(q_extended, I_noisy)
    porod_results[particle["name"]] = porod_result

    # Plot results
    plt.subplot(2, 3, i + 1)

    # Main plot: I vs q (log-log)
    plt.loglog(
        q_extended, I_noisy, "o", color=particle["color"], alpha=0.6, markersize=2
    )

    if porod_result and porod_result["valid_fit"]:
        plt.loglog(
            porod_result["q_fit"],
            porod_result["I_fit"],
            "-",
            color="black",
            linewidth=2,
            label="Porod fit",
        )

        # Mark Porod region
        q_min_porod = porod_result["q_range"][0]
        plt.axvline(
            q_min_porod,
            color="red",
            linestyle="--",
            alpha=0.7,
            label=f"qRg={porod_result['qRg_min']:.1f}",
        )

        # Show q⁻⁴ reference line
        q_ref = np.array([q_min_porod, q_extended.max()])
        I_ref = porod_result["K_porod"] / (q_ref**4)
        plt.loglog(q_ref, I_ref, ":", color="gray", alpha=0.7, label="q⁻⁴")

    plt.xlabel("q (Å⁻¹)")
    plt.ylabel("I(q)")
    plt.title(f"{particle['name']} - Porod Analysis")
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=8)

# Porod plots (q⁴I vs q)
for i, particle in enumerate(particles):
    R = particle["R"]

    # Generate data again (for consistency)
    q_extended = np.logspace(-2, 0.5, 80)
    F = sphere_form_factor(q_extended, R)
    I_clean = 1000 * F**2 + 2
    noise_level = 0.03
    I_noisy = I_clean * (1 + noise_level * np.random.normal(0, 1, len(q_extended)))
    I_noisy = np.maximum(I_noisy, 0.1)

    result = porod_results[particle["name"]]

    plt.subplot(2, 3, i + 4)

    # Porod plot: q⁴I vs q
    q4I = q_extended**4 * I_noisy
    plt.plot(q_extended, q4I, "o", color=particle["color"], alpha=0.6, markersize=2)

    if result and result["valid_fit"]:
        # Show Porod constant as horizontal line
        plt.axhline(
            result["K_porod"],
            color="black",
            linewidth=2,
            label=f"K = {result['K_porod']:.0f}",
        )

        # Mark analysis region
        q_min = result["q_range"][0]
        plt.axvline(q_min, color="red", linestyle="--", alpha=0.7)

        # Add text annotation
        plt.text(
            0.05,
            0.95,
            f"K = {result['K_porod']:.0f} ± {result['K_porod_error']:.0f}\n"
            f"Background = {result['background']:.1f}\n"
            f"χ²r = {result['chi2_reduced']:.2f}",
            transform=plt.gca().transAxes,
            fontsize=8,
            verticalalignment="top",
            bbox={"boxstyle": "round,pad=0.3", "facecolor": "white", "alpha": 0.8},
        )

    plt.xlabel("q (Å⁻¹)")
    plt.ylabel("q⁴I(q)")
    plt.title(f"Porod Plot - {particle['name']}")
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=8)

plt.tight_layout()
plt.show()

print("\n" + "=" * 50)
print("POROD ANALYSIS RESULTS")
print("=" * 50)

for particle in particles:
    result = porod_results[particle["name"]]
    print(f"\n{particle['name']}:")

    if result and result["valid_fit"]:
        print(
            f"  Porod constant K: {result['K_porod']:.0f} ± {result['K_porod_error']:.0f}"
        )
        print(
            f"  Background: {result['background']:.2f} ± {result['background_error']:.2f}"
        )
        print(
            f"  Q-range: {result['q_range'][0]:.3f} to {result['q_range'][1]:.3f} Å⁻¹"
        )
        print(f"  qRg_min: {result['qRg_min']:.2f}")
        print(f"  χ²_reduced: {result['chi2_reduced']:.2f}")

        if result["qRg_min"] < 3.0:
            print("  ⚠ Warning: qRg_min < 3.0 may indicate invalid Porod region")
        else:
            print("  ✓ Valid Porod region")
    else:
        print("  ❌ Porod fit failed")

print("\nPorod Analysis Notes:")
print("- The Porod constant K is proportional to surface area")
print("- For spheres: K = 2π(Δρ)²S, where S is total surface area")
print("- Valid Porod region typically requires qR > 3-5")
print("- Deviation from q⁻⁴ behavior may indicate surface roughness")

## Structure Factor Analysis

### Inter-particle Interactions

When particles interact, the total scattering becomes:
$$I(q) = n |F(q)|^2 S(q)$$

where S(q) is the structure factor describing correlations.

In [ ]:
def hard_sphere_structure_factor(q, R_hs, volume_fraction):
    """
    Hard sphere structure factor using Percus-Yevick closure.

    Parameters:
    -----------
    q : array
        Momentum transfer
    R_hs : float
        Hard sphere radius
    volume_fraction : float
        Volume fraction (0 < φ < 0.64 for spheres)

    Returns:
    --------
    S : array
        Structure factor values
    """
    phi = volume_fraction
    if phi >= 0.64:
        print("Warning: Volume fraction exceeds random close packing")

    # Percus-Yevick parameters
    alpha = (1 + 2 * phi) ** 2 / (1 - phi) ** 4
    beta = -6 * phi * (1 + phi / 2) ** 2 / (1 - phi) ** 4
    gamma = alpha * phi / 2

    # Dimensionless wavevector
    A = 2 * q * R_hs

    # Structure factor calculation
    S = np.zeros_like(q)

    for i, A_val in enumerate(A):
        if A_val < 1e-6:
            # Small q limit
            S[i] = 1 / (1 + 12 * phi * gamma)
        else:
            # Full calculation
            sin_A = np.sin(A_val)
            cos_A = np.cos(A_val)

            G = (
                alpha * (sin_A - A_val * cos_A) / A_val**2
                + beta * (2 * A_val * sin_A - (A_val**2 - 2) * cos_A - 2) / A_val**3
                + gamma
                * (
                    (
                        (4 * A_val**3 - 24 * A_val) * sin_A
                        - (A_val**4 - 12 * A_val**2 + 24) * cos_A
                        + 24
                    )
                    / A_val**5
                )
            )

            C = -24 * phi * G / A_val
            S[i] = 1 / (1 + C)

    return S


def yukawa_structure_factor(q, kappa, Z_eff):
    """
    Simplified Yukawa (screened Coulomb) structure factor.

    Parameters:
    -----------
    q : array
        Momentum transfer
    kappa : float
        Screening parameter (inverse Debye length)
    Z_eff : float
        Effective charge

    Returns:
    --------
    S : array
        Structure factor values
    """
    # Simplified expression for weak interactions
    S = 1 / (1 + Z_eff**2 / (q**2 + kappa**2))
    return S


# Generate examples with different interactions
q_sf = np.logspace(-2, 0.5, 100)
R_particle = 25  # Å

# Form factor (same for all)
F = sphere_form_factor(q_sf, R_particle)

# Different structure factors
S_dilute = np.ones_like(q_sf)  # No interactions
S_hs_low = hard_sphere_structure_factor(q_sf, R_particle, volume_fraction=0.1)
S_hs_high = hard_sphere_structure_factor(q_sf, R_particle, volume_fraction=0.3)
S_repulsive = yukawa_structure_factor(q_sf, kappa=0.05, Z_eff=10)

# Total intensities
I_dilute = 1000 * F**2 * S_dilute + 1
I_hs_low = 1000 * F**2 * S_hs_low + 1
I_hs_high = 1000 * F**2 * S_hs_high + 1
I_repulsive = 1000 * F**2 * S_repulsive + 1

plt.figure(figsize=(16, 12))

# Structure factors
plt.subplot(2, 3, 1)
plt.plot(q_sf, S_dilute, "k-", linewidth=2, label="Dilute (no interactions)")
plt.plot(q_sf, S_hs_low, "b-", linewidth=2, label="Hard spheres (φ=0.1)")
plt.plot(q_sf, S_hs_high, "r-", linewidth=2, label="Hard spheres (φ=0.3)")
plt.plot(q_sf, S_repulsive, "g-", linewidth=2, label="Electrostatic repulsion")
plt.xlabel("q (Å⁻¹)")
plt.ylabel("S(q)")
plt.title("Structure Factors")
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 3)

# Total intensities (log-log)
plt.subplot(2, 3, 2)
plt.loglog(q_sf, I_dilute, "k-", linewidth=2, label="Dilute")
plt.loglog(q_sf, I_hs_low, "b-", linewidth=2, label="HS (φ=0.1)")
plt.loglog(q_sf, I_hs_high, "r-", linewidth=2, label="HS (φ=0.3)")
plt.loglog(q_sf, I_repulsive, "g-", linewidth=2, label="Repulsive")
plt.xlabel("q (Å⁻¹)")
plt.ylabel("I(q)")
plt.title("Total Scattering Intensities")
plt.legend()
plt.grid(True, alpha=0.3)

# Small-q behavior (structure factor effects)
plt.subplot(2, 3, 3)
q_small = q_sf[q_sf <= 0.1]
plt.plot(q_small, S_dilute[: len(q_small)], "k-", linewidth=3, label="Dilute")
plt.plot(q_small, S_hs_low[: len(q_small)], "b-", linewidth=3, label="HS (φ=0.1)")
plt.plot(q_small, S_hs_high[: len(q_small)], "r-", linewidth=3, label="HS (φ=0.3)")
plt.plot(q_small, S_repulsive[: len(q_small)], "g-", linewidth=3, label="Repulsive")
plt.xlabel("q (Å⁻¹)")
plt.ylabel("S(q)")
plt.title("Small-q Structure Factor Behavior")
plt.legend()
plt.grid(True, alpha=0.3)

# Normalized intensities (remove form factor)
plt.subplot(2, 3, 4)
I_norm_dilute = I_dilute / (F**2 + 1e-10)
I_norm_hs_low = I_hs_low / (F**2 + 1e-10)
I_norm_hs_high = I_hs_high / (F**2 + 1e-10)
I_norm_repulsive = I_repulsive / (F**2 + 1e-10)

plt.plot(q_sf, I_norm_dilute, "k-", linewidth=2, label="Dilute")
plt.plot(q_sf, I_norm_hs_low, "b-", linewidth=2, label="HS (φ=0.1)")
plt.plot(q_sf, I_norm_hs_high, "r-", linewidth=2, label="HS (φ=0.3)")
plt.plot(q_sf, I_norm_repulsive, "g-", linewidth=2, label="Repulsive")
plt.xlabel("q (Å⁻¹)")
plt.ylabel("I(q) / |F(q)|²")
plt.title("Structure Factor from Normalized Intensity")
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 2000)

# Peak analysis for hard spheres
plt.subplot(2, 3, 5)
# Find structure factor peak position
peak_idx_low = np.argmax(S_hs_low)
peak_idx_high = np.argmax(S_hs_high)
q_peak_low = q_sf[peak_idx_low]
q_peak_high = q_sf[peak_idx_high]

plt.plot(q_sf, S_hs_low, "b-", linewidth=2, label=f"φ=0.1, q_peak={q_peak_low:.3f}")
plt.plot(q_sf, S_hs_high, "r-", linewidth=2, label=f"φ=0.3, q_peak={q_peak_high:.3f}")
plt.axvline(q_peak_low, color="blue", linestyle="--", alpha=0.7)
plt.axvline(q_peak_high, color="red", linestyle="--", alpha=0.7)
plt.xlabel("q (Å⁻¹)")
plt.ylabel("S(q)")
plt.title("Hard Sphere Peak Analysis")
plt.legend()
plt.grid(True, alpha=0.3)

# Calculate average nearest-neighbor distances
d_nn_low = 2 * np.pi / q_peak_low
d_nn_high = 2 * np.pi / q_peak_high

plt.text(
    0.5,
    0.8,
    f"Nearest-neighbor distances:\n"
    f"φ=0.1: d = {d_nn_low:.1f} Å\n"
    f"φ=0.3: d = {d_nn_high:.1f} Å\n"
    f"Particle diameter: {2 * R_particle} Å",
    transform=plt.gca().transAxes,
    fontsize=9,
    bbox={"boxstyle": "round,pad=0.5", "facecolor": "lightblue", "alpha": 0.8},
)

# Summary table
plt.subplot(2, 3, 6)
plt.axis("off")

summary_text = f"""
STRUCTURE FACTOR ANALYSIS
{"=" * 30}

System Parameters:
  Particle radius: {R_particle} Å
  Particle diameter: {2 * R_particle} Å

Hard Sphere Results:
  φ = 0.1:
    Peak position: {q_peak_low:.3f} Å⁻¹
    Peak height: {S_hs_low[peak_idx_low]:.2f}
    d_nn: {d_nn_low:.1f} Å

  φ = 0.3:
    Peak position: {q_peak_high:.3f} Å⁻¹
    Peak height: {S_hs_high[peak_idx_high]:.2f}
    d_nn: {d_nn_high:.1f} Å

Physical Insights:
  • Higher φ → stronger correlations
  • Peak shifts to higher q
  • Peak height increases
  • Shorter nearest-neighbor distance
"""

plt.text(
    0.05,
    0.95,
    summary_text,
    transform=plt.gca().transAxes,
    fontsize=9,
    verticalalignment="top",
    fontfamily="monospace",
    bbox={"boxstyle": "round,pad=0.5", "facecolor": "lightyellow", "alpha": 0.9},
)

plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("STRUCTURE FACTOR ANALYSIS RESULTS")
print("=" * 60)

print("\nHard Sphere Analysis:")
print("  Volume fraction φ = 0.1:")
print(f"    Structure factor peak: q = {q_peak_low:.3f} Å⁻¹")
print(f"    Peak height: S(q_max) = {S_hs_low[peak_idx_low]:.2f}")
print(f"    Nearest-neighbor distance: {d_nn_low:.1f} Å")
print(f"    Ratio d_nn/(2R): {d_nn_low / (2 * R_particle):.2f}")

print("\n  Volume fraction φ = 0.3:")
print(f"    Structure factor peak: q = {q_peak_high:.3f} Å⁻¹")
print(f"    Peak height: S(q_max) = {S_hs_high[peak_idx_high]:.2f}")
print(f"    Nearest-neighbor distance: {d_nn_high:.1f} Å")
print(f"    Ratio d_nn/(2R): {d_nn_high / (2 * R_particle):.2f}")

print("\nStructure Factor Interpretation:")
print("  • S(q→0) < 1: Repulsive interactions suppress density fluctuations")
print("  • S(q) peak: Indicates preferred inter-particle spacing")
print("  • Peak position: q_max ≈ 2π/d_nn (nearest-neighbor distance)")
print("  • Peak intensity: Increases with concentration and interaction strength")
print("  • For hard spheres: d_nn > 2R (excluded volume effect)")

## Working with 2D SAXS Data

### Anisotropic Scattering Analysis

In [ ]:
# Generate synthetic 2D SAXS pattern
def generate_2d_pattern(size=256, center=None, pattern_type="isotropic"):
    """
    Generate synthetic 2D SAXS patterns.

    Parameters:
    -----------
    size : int
        Image size (pixels)
    center : tuple
        Beam center (x, y). If None, uses image center
    pattern_type : str
        'isotropic', 'fiber', 'powder_rings', or 'oriented'

    Returns:
    --------
    pattern : 2D array
        Scattering intensity pattern
    q_x, q_y : 2D arrays
        Momentum transfer components
    """
    if center is None:
        center = (size // 2, size // 2)

    # Create coordinate grids
    x = np.arange(size) - center[0]
    y = np.arange(size) - center[1]
    X, Y = np.meshgrid(x, y)

    # Convert to q-space (arbitrary units)
    pixel_size = 0.001  # Å⁻¹ per pixel
    q_x = X * pixel_size
    q_y = Y * pixel_size
    q_r = np.sqrt(q_x**2 + q_y**2)
    q_phi = np.arctan2(q_y, q_x)

    if pattern_type == "isotropic":
        # Simple isotropic scattering (sphere-like)
        R = 50  # Effective size
        F = sphere_form_factor(q_r, R)
        pattern = 1000 * F**2 * np.exp(-((q_r / 0.3) ** 2)) + 10

    elif pattern_type == "fiber":
        # Fiber-like pattern (oriented cylinders)
        # Intensity depends on angle
        orientation_factor = np.cos(q_phi) ** 2  # Prefer equatorial direction
        R = 20
        L = 200
        F = cylinder_form_factor(q_r, R, L)
        pattern = 1000 * F**2 * orientation_factor * np.exp(-((q_r / 0.2) ** 2)) + 5

    elif pattern_type == "powder_rings":
        # Powder diffraction rings
        ring_positions = [0.05, 0.08, 0.12, 0.16]  # Å⁻¹
        ring_width = 0.005
        pattern = np.ones_like(q_r) * 5  # Background

        for ring_q in ring_positions:
            ring_intensity = 500 * np.exp(-(((q_r - ring_q) / ring_width) ** 2))
            pattern += ring_intensity

    elif pattern_type == "oriented":
        # Oriented domains with preferred direction
        preferred_angle = np.pi / 4  # 45 degrees
        angular_width = np.pi / 8  # ±22.5 degrees

        angle_factor = np.exp(-(((q_phi - preferred_angle) / angular_width) ** 2))
        angle_factor += np.exp(
            -(((q_phi - preferred_angle + np.pi) / angular_width) ** 2)
        )  # Symmetric

        R = 30
        F = sphere_form_factor(q_r, R)
        pattern = 800 * F**2 * angle_factor * np.exp(-((q_r / 0.25) ** 2)) + 8

    # Add noise
    noise = np.random.poisson(pattern * 0.1) * 0.1
    pattern += noise

    return pattern, q_x, q_y


# Generate different 2D patterns
patterns = {
    "Isotropic": generate_2d_pattern(pattern_type="isotropic"),
    "Fiber": generate_2d_pattern(pattern_type="fiber"),
    "Powder Rings": generate_2d_pattern(pattern_type="powder_rings"),
    "Oriented Domains": generate_2d_pattern(pattern_type="oriented"),
}

# Display 2D patterns
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i, (name, (pattern, _q_x, _q_y)) in enumerate(patterns.items()):
    # 2D pattern (linear scale)
    ax1 = axes[0, i]
    im1 = ax1.imshow(
        pattern, origin="lower", cmap="hot", vmin=0, vmax=np.percentile(pattern, 99)
    )
    ax1.set_title(f"{name}\n(Linear Scale)")
    ax1.set_xlabel("X (pixels)")
    ax1.set_ylabel("Y (pixels)")

    # Add beam center marker
    center = (pattern.shape[1] // 2, pattern.shape[0] // 2)
    ax1.plot(center[0], center[1], "w+", markersize=10, markeredgewidth=2)

    plt.colorbar(im1, ax=ax1, shrink=0.8)

    # 2D pattern (log scale)
    ax2 = axes[1, i]
    pattern_log = np.log10(np.maximum(pattern, 1))
    im2 = ax2.imshow(pattern_log, origin="lower", cmap="hot")
    ax2.set_title(f"{name}\n(Log Scale)")
    ax2.set_xlabel("X (pixels)")
    ax2.set_ylabel("Y (pixels)")
    ax2.plot(center[0], center[1], "w+", markersize=10, markeredgewidth=2)
    plt.colorbar(im2, ax=ax2, shrink=0.8)

plt.tight_layout()
plt.show()

print("2D SAXS Pattern Types:")
print("• Isotropic: Spherical particles, random orientation")
print("• Fiber: Oriented rod-like structures, preferred alignment")
print("• Powder Rings: Crystalline material, random grain orientation")
print("• Oriented Domains: Anisotropic structures with preferred directions")

# 2. Basic visualization
saxs1d.pg_plot(xf_list=[xf], pg_hdl=matplotlib_handle, plot_type=3)